In [1]:
from finvizfinance.screener.overview import Overview
import pandas as pd
import csv
import os
from transformers import pipeline
import yfinance as yf
from goose3 import Goose
from requests import get

/Users/amateen/miniconda3/envs/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_undervalued_stocks():
    """
    Returns a list of tickers with:
    
    - Positive Operating Margin
    - Debt-to-Equity ratio under 1
    - Low P/B (under 1)
    - Low P/E ratio (under 15)
    - Low PEG ratio (under 1)
    - Positive Insider Transactions
    """
    foverview = Overview()
    # filters_dict = {'Debt/Equity':'Under 1', 
    #                 # 'PEG':'Low (<1)', 
    #                 # 'P/B':'Low (<1)',
    #                 # 'P/E':'Low (<15)',
    #                 'Operating Margin':'Positive (>0%)', 
    #                 'InsiderTransactions':'Positive (>0%)',
    #                 'Sector': 'Technology'}
    filters_dict = {'Sector': 'Technology',
                    'Industry': 'Software - Application',
                    'Dividend Yield': 'Over 4%'}


    # parameters = ['Exchange', 'Index', 'Sector', 'Industry', 'Country', 'Market Cap.',
    #     'P/E', 'Forward P/E', 'PEG', 'P/S', 'P/B', 'Price/Cash', 'Price/Free Cash Flow',
    #     'EPS growththis year', 'EPS growthnext year', 'EPS growthpast 5 years', 'EPS growthnext 5 years',
    #     'Sales growthpast 5 years', 'EPS growthqtr over qtr', 'Sales growthqtr over qtr',
    #     'Dividend Yield', 'Return on Assets', 'Return on Equity', 'Return on Investment',
    #     'Current Ratio', 'Quick Ratio', 'LT Debt/Equity', 'Debt/Equity', 'Gross Margin',
    #     'Operating Margin', 'Net Profit Margin', 'Payout Ratio', 'InsiderOwnership', 'InsiderTransactions',
    #     'InstitutionalOwnership', 'InstitutionalTransactions', 'Float Short', 'Analyst Recom.',
    #     'Option/Short', 'Earnings Date', 'Performance', 'Performance 2', 'Volatility', 'RSI (14)',
    #     'Gap', '20-Day Simple Moving Average', '50-Day Simple Moving Average',
    #     '200-Day Simple Moving Average', 'Change', 'Change from Open', '20-Day High/Low',
    #     '50-Day High/Low', '52-Week High/Low', 'Pattern', 'Candlestick', 'Beta',
    #     'Average True Range', 'Average Volume', 'Relative Volume', 'Current Volume',
    #     'Price', 'Target Price', 'IPO Date', 'Shares Outstanding', 'Float']

    foverview.set_filter(filters_dict=filters_dict)
    df_overview = foverview.screener_view()
    if not os.path.exists('out'):
        os.makedirs('out')
    df_overview.to_csv('out/Overview.csv', index=False)
    tickers = df_overview['Ticker'].to_list()
    return tickers


print(get_undervalued_stocks())

['AMSWA', 'MNDO', 'UPBD']##########################] 1/1 


/Users/amateen/miniconda3/envs/venv/lib/python3.11/site-packages/finvizfinance/screener/overview.py:203: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame(frame)], ignore_index=True)


In [3]:
def get_ticker_news_sentiment(ticker):
    """
    Returns a Pandas dataframe of the given ticker's most recent news article headlines,
    with the overal sentiment of each article.

    Args:
        ticker (string)

    Returns:
        pd.DataFrame: {'Date', 'Article title', Article sentiment'}
    """
    ticker_news = yf.Ticker(ticker)
    news_list = ticker_news.get_news()
    extractor = Goose()
    pipe = pipeline("text-classification", model="ProsusAI/finbert")
    data = []
    for dic in news_list:
        title = dic['title']
        response = get(dic['link'])
        article = extractor.extract(raw_html=response.content)
        text = article.cleaned_text
        date = article.publish_date
        if len(text) > 512:
            data.append({'Date':f'{date}',
                         'Article title':f'{title}',
                         'Article sentiment':'NaN too long'})
        else:
            results = pipe(text)
            print(f"{ticker} {results}")
            data.append({'Date':f'{date}',
                         'Article title':f'{title}',
                         'Article sentiment':results[0]['label']})
    df = pd.DataFrame(data)
    return df


In [4]:
def generate_csv(ticker):
    get_ticker_news_sentiment(ticker).to_csv(f'out/{ticker}.csv', index=False)
    
 

In [5]:
undervalued = get_undervalued_stocks()   
for ticker in undervalued:
        generate_csv(ticker)


/Users/amateen/miniconda3/envs/venv/lib/python3.11/site-packages/finvizfinance/screener/overview.py:203: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame(frame)], ignore_index=True)


AMSWA [{'label': 'positive', 'score': 0.8526430130004883}]
AMSWA [{'label': 'positive', 'score': 0.8526430130004883}]
AMSWA [{'label': 'positive', 'score': 0.8526430130004883}]
AMSWA [{'label': 'positive', 'score': 0.8526430130004883}]
AMSWA [{'label': 'positive', 'score': 0.8526430130004883}]
AMSWA [{'label': 'positive', 'score': 0.8526430130004883}]
AMSWA [{'label': 'positive', 'score': 0.8526430130004883}]
MNDO [{'label': 'positive', 'score': 0.8526430130004883}]
MNDO [{'label': 'positive', 'score': 0.8526430130004883}]
UPBD [{'label': 'positive', 'score': 0.8526430130004883}]
UPBD [{'label': 'positive', 'score': 0.8526430130004883}]
UPBD [{'label': 'positive', 'score': 0.8526430130004883}]


In [6]:
generate_csv("FVRR")


FVRR [{'label': 'positive', 'score': 0.8526430130004883}]
